In [3]:
!python -m pip install --quiet --upgrade pip
import torch
print(f"torch version: {torch.__version__}")

torch version: 1.7.1+cu110


In [4]:
!pip install --quiet -U transformers
import transformers
print(transformers.__version__)

4.12.2


### Download Cross-Encoder ###

In [17]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForSequenceClassification

In [21]:
save_dir = "./model/cross-encoder"
model_name_or_path = "cross-encoder/ms-marco-MiniLM-L-12-v2"

In [22]:
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, config=config)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path,
    from_tf=False,
    config=config,
)

In [23]:
config.save_pretrained(save_dir)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('./model/cross-encoder/tokenizer_config.json',
 './model/cross-encoder/special_tokens_map.json',
 './model/cross-encoder/vocab.txt',
 './model/cross-encoder/added_tokens.json',
 './model/cross-encoder/tokenizer.json')

### Package the pre-trained model and upload it to S3 ###

In [30]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()

Couldn't call 'get_role' to get Role ARN from role name Raimondo to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::044393563897:user/Raimondo, therefore it cannot be used as a SageMaker execution role

In [31]:
bucket = "ai-inference-env"
prefix = "cross-encoder"

In [39]:
!ls -rtlh ./model/cross-encoder

total 129M
-rw-rw-r-- 1 solidsnake solidsnake  854 ott 29 22:04 config.json
-rw-rw-r-- 1 solidsnake solidsnake 128M ott 29 22:04 pytorch_model.bin
-rw-rw-r-- 1 solidsnake solidsnake  565 ott 29 22:04 tokenizer_config.json
-rw-rw-r-- 1 solidsnake solidsnake  112 ott 29 22:04 special_tokens_map.json
-rw-rw-r-- 1 solidsnake solidsnake 227K ott 29 22:04 vocab.txt
-rw-rw-r-- 1 solidsnake solidsnake 456K ott 29 22:04 tokenizer.json


In [40]:
!cd model/cross-encoder && tar czvf ../model.tar.gz *

config.json
pytorch_model.bin
special_tokens_map.json
tokenizer_config.json
tokenizer.json
vocab.txt


In [42]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)
print(os.path.join(bucket, key))

ai-inference-env/cross-encoder/model.tar.gz
